# 🎯 Project Brief
At Mappa, we're redefining the experience of finding a new home. From finding a new neighbourhood, to property viewings, negotiations and dealing with admin (e.g. mortgage and legal), your decision-making potential will be maximised to allow you secure the best value for your money.

Our mission is to hand back knowledge and control to the house hunter using the power of data and technology. Our goal with this project is create is to create a master list of listed properties starting with London. Your task is the extract and blend property datasets across the Greater London area (United Kingdom) in one dataset.

**Key Deliverables:**
* Code base 
* Documentation
* Architecture diagram of flow

**Review and Selection:**
Contestants will be evaluated on:
* End-to-end process
* Code base quality
* Extracted dataset (output) quality
* Delivery and Reporting

Design thinking is important. Contestants can go to any length to provide an end to end solution that is fully automated, tested and data could be pulled from a database.

**🛠️ Tools & Resources**
* Python
* Scraper API

### 👣 Process
* Extract / scrape the datasets from all the above websites for a given day of your choosing. Please be aware that:
 * Individual sources may not offer the same coverage quality and consistency of the fields listed earlier
 * datasets for rental properties and properties for sale will be somewhat different—it is usually advisable to keep them as separate tables.
* Blend these data sets together into one dataset with unique records:
 * Devise a way to check for duplicate records within the individual datasets then de-dupe
 * Identify/create a unique identifier that can be used to combine the various records across the datasets
 * Use that Unique ID to create a Master list of London properties

Please be aware that creating the aggregated unique list is a key challenge:
* Agents will typically list a single property on multiple websites (Some properties will appear on Zoopla, Rightmove and OTM)
* Some agents may use one listing site but not the other (i.e. some property records will exist on Zoopla but not Rightmove and vice versa)

### 🔖 About the Data
**Property sites**
* [Rightmove](https://www.rightmove.co.uk)
* [Zoopla](https://www.zoopla.co.uk)
* [On The Market (OTM)](https://www.onthemarket.com)

**Required fields**
* Transaction type (i.e. sale vs. rent - string)
* Bedrooms (integer)
* Bathrooms (integer)
* Description (free text string)
* Property type e.g. flat, detached house, terraced house
* Price e.g. 500,000 (typically integer)
* Location: Key location data here is Postcode district and/or Postcode
* Agent (advertising the property)
* Listing source
* Listing URL
* Other nice-to-have metadata, e.g. If a rental property is furnished or not
* Anything else you deem interesting

**Geographical remit**
Greater London

### Collect & Load Data

In [28]:
import re
import datetime

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

We've collected the data from the sites. We'll clean them to ensure they reflect the relevant postcodes and the information is clear.

In [2]:
postcodes = pd.read_excel('London postcode districts.xlsx')
postcodes.head()

,Postcode district,Local Areas,Borough
0,BR1,"Bromley, Bickley, Downham",Bromley
1,BR2,"Bickley, Hayes, Bromley Common, Shortlands",Bromley
2,BR3,"Beckenham, Eden Park, Elmers End, Park Langley...",Bromley
3,BR4,West Wickham,Bromley
4,BR5,"Orpington, St Mary Cray, Petts Wood",Bromley


In [71]:
rm_rent = pd.read_csv('rm_rent_df.csv')
rm_sale = pd.read_csv('rm_sale_df.csv')
otm_rent = pd.read_csv('otm_rent_df.csv')
otm_sale = pd.read_csv('otm_sale_df.csv')
zp_rent = pd.read_csv('zp_rent_df.csv')
zp_sale = pd.read_csv('zp_sale_df.csv')

In [72]:
rm_rent = rm_rent.assign(
    postcode=lambda x: x.address.str.split(' ').str[-1],
    listing_source='rightmove',
)

rm_sale = rm_sale.assign(
    postcode=lambda x: x.address.str.split(' ').str[-1],
    listing_source='rightmove',
)

otm_rent = otm_rent.assign(
    postcode=lambda x: x.address.str.split(' ').str[-1],
    listing_source='onthemarket',
)

otm_sale = otm_sale.assign(
    postcode=lambda x: x.address.str.split(' ').str[-1],
    listing_source='onthemarket',
)

zp_rent = zp_rent.assign(
    postcode=lambda x: x.address.str.split(' ').str[-1],
    listing_source='zoopla',
)

zp_sale = zp_sale.assign(
    postcode=lambda x: x.address.str.split(' ').str[-1],
    listing_source='zoopla',
)

In [73]:
rm_rent.head()

,property_id,title,url,address,country,tag,property_status,primary price,secondary price,agent,...,date_added,letting_details,property_type,beds,baths,key_features,description,property_size,postcode,listing_source
0,prop139453520,\n 2 bedroom flat,https://www.rightmove.co.uk/properties/1394535...,"Regents Park Road, Primrose Hill NW1",GB,Commercial,\nDevelopment Microsite\n\n\n\n\n\n,"£3,120 pcm",£720 pwThe amount per month or week you need t...,"Parkheath, Belsize Park",...,Added on 04/09/2023,"['Let available date: 29/10/2023', 'Deposit: £...",Flat,×2,×2,"['Characterful 2 bedroom apartment', 'Private ...",Bright and characterful 2 bedroom apartment -...,912 sq. ft.,NW1,rightmove
1,prop138393464,\n 3 bedroom apartment,https://www.rightmove.co.uk/properties/1383934...,"Bolsover Street, London, W1W",GB,Commercial,\nDevelopment Microsite\n\n\n\n\n\n,"£7,800 pcm","£1,800 pwThe amount per month or week you need...","AC RESIDENTIAL, Mayfair",...,Reduced today,"['Let available date: Now', 'Deposit: £10,800A...",Apartment,×3,×3,"['Fully Furnished', 'Shops and amenities nearb...",THE HOMEFully renovated and interior-designed ...,NaN,W1W,rightmove
2,prop139667669,\n 2 bedroom ground floor flat,https://www.rightmove.co.uk/properties/1396676...,Prince Of Wales Terrace,GB,Commercial,\nDevelopment Microsite\n\n\n\n\n\n,"£13,000 pcm","£3,000 pwThe amount per month or week you need...","Scraye, London",...,Added today,"['Let available date: Now', 'Deposit: Ask agen...",Ground Flat,×2,×2,"['Terrace', 'Short lets available', 'Elevator'...",Reference: 9585#Presenting a newly-refurbished...,"1,035 sq. ft.",Terrace,rightmove
3,prop139667666,\n 2 bedroom flat,https://www.rightmove.co.uk/properties/1396676...,Rainville Road,GB,Commercial,\nDevelopment Microsite\n\n\n\n\n\n,"£3,878 pcm",£895 pwThe amount per month or week you need t...,"Scraye, London",...,Added today,"['Let available date: Now', 'Deposit: Ask agen...",Flat,×2,×1,"['24 hour security', 'Short lets available', '...",Reference: 9760#No Deposit available. Presenti...,667 sq. ft.,Road,rightmove
4,prop139667426,\n 2 bedroom apartment,https://www.rightmove.co.uk/properties/1396674...,"Heath Road, Weybridge, Surrey, KT13",GB,Commercial,\nDevelopment Microsite\n\n\n\n\n\n,"£1,750 pcm",£404 pwThe amount per month or week you need t...,"Winkworth, Weybridge Lettings",...,Added today,"['Let available date: 15/09/2023', 'Deposit: £...",Apartment,×2,×1,NaN,A newly refurbished 1st floor purpose built ap...,NaN,KT13,rightmove


### Data Cleaning
1. Merge dataframes
2. Filter dataframes to include only postcodes in the `postcodes` dataframe
3. Remove trailing spaces
4. Drop irrelevant columns
5. Clean and convert columns to appropriate datatype
6. Clean and create new columns where necessary

**Merge Dataframes:**

In [115]:
rent_df = pd.concat([rm_rent, otm_rent, zp_rent], axis=0, ignore_index=True).rename(
    columns={'primary price': 'rent_per_month',
             'secondary price': 'rent_per_week'}
).assign(
    listing_type='rent'
)

rent_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2631 entries, 0 to 2630
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   property_id      2631 non-null   object 
 1   title            2622 non-null   object 
 2   url              2631 non-null   object 
 3   address          2622 non-null   object 
 4   country          1050 non-null   object 
 5   tag              1050 non-null   object 
 6   property_status  1955 non-null   object 
 7   rent_per_month   2604 non-null   object 
 8   rent_per_week    1699 non-null   object 
 9   agent            2622 non-null   object 
 10  agent_address    1972 non-null   object 
 11  agent_phone      1972 non-null   object 
 12  agent_url        2621 non-null   object 
 13  date_added       2631 non-null   object 
 14  letting_details  1699 non-null   object 
 15  property_type    1973 non-null   object 
 16  beds             2476 non-null   object 
 17  baths         

In [116]:
sale_df = pd.concat([rm_sale, otm_sale, zp_sale], axis=0, ignore_index=True).assign(
    listing_type='sale'
)

sale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2583 entries, 0 to 2582
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   property_id      2583 non-null   object
 1   title            2579 non-null   object
 2   url              2583 non-null   object
 3   address          2579 non-null   object
 4   tag              1050 non-null   object
 5   property_status  1927 non-null   object
 6   country          1050 non-null   object
 7   price            2574 non-null   object
 8   agent            2579 non-null   object
 9   agent_address    1932 non-null   object
 10  agent_phone      1932 non-null   object
 11  agent_url        2577 non-null   object
 12  date_added       2583 non-null   object
 13  property_type    1932 non-null   object
 14  beds             2558 non-null   object
 15  baths            2476 non-null   object
 16  property_size    972 non-null    object
 17  tenure           1041 non-null   

**Drop Missing Values**

In [117]:
missing_values_count = rent_df.isna().sum(axis=1)
rent_df = rent_df.drop(rent_df[missing_values_count > 13].index)

In [118]:
missing_values_count = sale_df.isna().sum(axis=1)
sale_df = sale_df.drop(sale_df[missing_values_count > 14].index)

**Filter Dataframes by Postcode**

In [119]:
areas = postcodes['Local Areas'].str.split(', ')
rent_areas = rent_df.query('postcode.str.len() > 4').address.str.split(', ')
# rent_areas = pd.concat([rent_areas, areas], axis=1)

In [132]:
def check_value_presence(ser1, ser2):
    # Check if any value in list1 is present in list2
    val_ind = list()
    for ind, list1 in ser1.items():
        for ind2, list2 in ser2.items():
            for item in list1:
                if isinstance(item, str) and item in list2:
                    val_ind.append([ind, ind2]) 
    return val_ind

indices = check_value_presence(rent_areas, areas)

In [131]:
rent_areas.loc[322], areas.loc[248]

(['Old Oak Common Lane', 'Acton'],
 ['Acton', 'East Acton', 'Park Royal', 'West Acton'])

In [129]:
postcodes.loc[[115, 248]]

,Postcode district,Local Areas,Borough
115,NW10,"Willesden, Acton, Harlesden, Kensal Green, Nea...",Brent
248,W3,"Acton, East Acton, Park Royal, West Acton",Ealing


In [ ]:
rent_df.query('postcode.str.len() > 4').address.str.split(',')

In [ ]:
postcodes.loc[postcodes['Local Areas'].str.contains('Station Road', re.I)]

In [ ]:
postcodes.loc[postcodes['Borough'].str.contains('Station Road', re.I)]

In [ ]:
rent_df.loc[rent_df.postcode.isin(postcodes['Postcode district'])]